# Import Training Data, Add Labels, Check

In [7]:
#add labels, print dataframes to check

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
train_map = open("train.map")
train_label = open("train.label")
train_data = open("train.data")
train_data = open("train.data")

train_map = pd.read_csv(train_map, sep = ' ', names =['labelName' , 'labelId'])
train_map =train_map.reset_index()

train_label = pd.read_csv(train_label, names = ['label_id'])
train_label = train_label.reset_index()
train_label.rename(columns = {'index':'docIdx'}, inplace= True)

#add one to the doc ID so that there are no zeroes
train_label['docIdx'] = train_label['docIdx']+1

train_data_df = pd.read_csv(train_data, sep = ' ', names=['docIdx','wordIdx','count'])
train_map.head()

,index,labelName,labelId
0,0,alt.atheism,1
1,1,comp.graphics,2
2,2,comp.os.ms-windows.misc,3
3,3,comp.sys.ibm.pc.hardware,4
4,4,comp.sys.mac.hardware,5


In [8]:
train_label.head()

,docIdx,label_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [1]:
!pwd

/Users/ryanriopelle/Documents/DSE/DSE210/hw_2/Processed Version/NaiveBayes


In [6]:
train_data_df.head()

,docIdx,wordIdx,count
0,1,1,4
1,1,2,2
2,1,3,10
3,1,4,4
4,1,5,2


# Load Entire Vocabulary, Set Index For Every Word in Vocab

In [11]:
#This is necesssary so that the entire vocabulary can be referenced
vocabulary_data = open("vocabulary.txt")
vocabulary_df = pd.read_csv(vocabulary_data, names =['word'])
vocabulary_df = vocabulary_df.reset_index()
vocabulary_df.rename(columns = {'index':'wordIdx'}, inplace= True)
#Had to add 1 because word index starts at zero
vocabulary_df['wordIdx'] = vocabulary_df['wordIdx']+1
vocabulary_df = vocabulary_df.reset_index()
vocabulary_df = vocabulary_df.set_index('word')
vocabulary_df.head()

,index,wordIdx
word,,
archive,0,1
name,1,2
atheism,2,3
resources,3,4
alt,4,5


# Removed Stopwords from NLTK 1(f)
#This portion of the code removes words that do not help the algorythm in sorting, it is done to increase performance by reducing the size of the vocab overall

In [14]:
import nltk
from nltk.corpus import stopwords
stop_words = []
stop_words = stopwords.words('english')
vocabulary_df = vocabulary_df[(~vocabulary_df.index.isin(stop_words))]
vocabulary_df.reset_index(inplace= True)
vocabulary_df.shape

(61044, 3)

# Merge Training Data Tables

In [17]:
train_data_merged = pd.merge(train_label, train_data_df,on = 'docIdx', how='inner')
train_data_merged.head()

,docIdx,label_id,wordIdx,count
0,1,1,1,4
1,1,1,2,2
2,1,1,3,10
3,1,1,4,4
4,1,1,5,2


# Calculate Prior Probability

# Get total document count by searching the unique values, this could also be done by taking max of the ID 

In [21]:
# Get total document count by searching the unique values
count_of_doc = train_data_merged['docIdx'].unique()
no_of_doc = count_of_doc.shape
no_of_doc

(11269,)

In [22]:
# Prior Probability py(J) = No of Uniqe Document Ids in Label / Total no of Documents in Label
df_grouped = train_data_merged.groupby('label_id').docIdx.nunique()/no_of_doc
df_classprobability = pd.DataFrame(df_grouped)
df_classprobability

,docIdx
label_id,
1,0.042595
2,0.051557
3,0.050759
4,0.052090
5,0.051025
6,0.052533
7,0.051646
8,0.052533
9,0.052888


# Getting the Log of the Class probability 

In [23]:
df_classprobability.rename(columns = {'docIdx':'class probability'}, inplace= True)
df_classprobability['log class probability'] = np.log(df_classprobability)
df_classprobability.head()

,class probability,log class probability
label_id,,
1,0.042595,-3.156025
2,0.051557,-2.965060
3,0.050759,-2.980672
4,0.052090,-2.954786
5,0.051025,-2.975441


# Set Up Training Data Probability Distribution
Word Frequency for each word

In [27]:
# Count of a specific word in the same class/total number of words in data
# Sum of words in all the Documents for given label.
df_word_count = train_data_merged.groupby(['label_id'])[['count']].sum().reset_index()
df_word_count.head()

,label_id,count
0,1,148812
1,2,110358
2,3,90767
3,4,99146
4,5,86190


In [28]:
# Sum of word in all the documents for given label and WordIdx .
df_word_frequency = train_data_merged.groupby(['label_id','wordIdx'])[['count']].sum().reset_index()
df_word_frequency.rename(columns = {'count':'word_count'}, inplace= True)
df_word_frequency.head(3)

,label_id,wordIdx,word_count
0,1,1,13
1,1,2,63
2,1,3,275


In [31]:
#Merge the vocabulory table to have all words in table and assign zero for word count for the missing ones
def group_merge_vocab(grp):
    return grp.merge(vocabulary_df, how='right').fillna(0).sort('wordIdx')

# Groupby label ID for word frequency
df_word_freq = df_word_frequency.groupby('label_id').apply(group_merge_vocab)[['wordIdx','word_count']]
df_word_freq.reset_index(inplace= True)
df_word_freq.head()

,label_id,level_1,wordIdx,word_count
0,1,0,1,13
1,1,1,2,63
2,1,2,3,275
3,1,3,4,9
4,1,4,5,82


# Get word frequency for each label

In [32]:
distribution =  pd.merge(df_word_freq, df_word_count,on = 'label_id', how='inner')
distribution.rename(columns = {'count':'word_freq'}, inplace= True)
distribution.head()

,label_id,level_1,wordIdx,word_count,word_freq
0,1,0,1,13,148812
1,1,1,2,63,148812
2,1,2,3,275,148812
3,1,3,4,9,148812
4,1,4,5,82,148812


# Prior Probability Calculation for P(j)

In [34]:
# Get the log of the prior probability along with the prior prob
distribution['prior probability'] =  (distribution['word_count'] + 1)/(distribution['word_freq'] + len(vocabulary_df))
train_prob = distribution[['label_id','wordIdx','prior probability']]
train_prob['log prior probability'] = np.log(train_prob['prior probability'])
train_prob.head()

,label_id,wordIdx,prior probability,log prior probability
0,1,1,0.000067,-9.615120
1,1,2,0.000305,-8.095294
2,1,3,0.001315,-6.633776
3,1,4,0.000048,-9.951592
4,1,5,0.000396,-7.835336


# Import Test Data and Load in DF

In [42]:
test_data = open("test.data")
df_test_data = pd.read_csv(test_data, sep = ' ', names=['t_docIdx','wordIdx','t_count'])
test_label = open("test.label")
test_labels = pd.read_csv(test_label, sep = ' ', names=['t_labelIdx'])
test_labels = test_labels.reset_index()
test_labels.rename(columns = {'index':'t_docIdx'}, inplace= True)
test_labels['t_docIdx'] = test_labels['t_docIdx'] + 1
df_test_data.head(2)

,t_docIdx,wordIdx,t_count
0,1,3,1
1,1,10,1


In [39]:
# Load test label in the dataframe
test_labels.head(3)

,t_docIdx,t_labelIdx
0,1,1
1,2,1
2,3,1


# Setup Bayesian Algorithm: Map Probability for a Doc that Belongs to Each Class
#1) Match on Doc ID, #2 Pull Out Label ID, Probability Word Belongs to that class, and count of word occurence in document; sort by word ID so you can see the map of each word mapping to each class #3 multiply words to get prior probability #4 sum prior probability over each class (it can be added due to the log). #5 raname so that ID's line up and pie values for each class can be multiplied #6 add up the pi with the prior prob, can add because of the log values #7 Answer gives the probability that the document belongs to each class
#Log(Probability(word/class)^N,words)summed

In [43]:
def Naive_Bayes_Classifier(df, log):
    df_NaiveBayes = pd.merge(train_prob, df, on = ['wordIdx'], how='inner' )
    if log == False:
        df_NaiveBayes['test word prob'] = df_NaiveBayes['log prior probability']*df_NaiveBayes['t_count']
    else:
        df_NaiveBayes['test word prob'] = df_NaiveBayes['log prior probability']*(np.log(1+df_NaiveBayes['t_count']))
    Naive_Bayes_word_prob = pd.DataFrame(df_NaiveBayes.groupby(['label_id','t_docIdx'])['test word prob'].sum())
    Naive_Bayes_word_prob = Naive_Bayes_word_prob.join(df_classprobability)
    Naive_Bayes_word_prob['predicted prob for doc'] = Naive_Bayes_word_prob[['test word prob','log class probability']].sum(axis=1)
    Naive_Bayes_word_prob = Naive_Bayes_word_prob.reset_index()
    df_NaiveBayes_predicted = Naive_Bayes_word_prob.sort(['t_docIdx', 'predicted prob for doc'],ascending=[True,False]).groupby('t_docIdx').first()
    df_NaiveBayes_predicted.reset_index(inplace= True)
    classify = pd.merge(test_labels, df_NaiveBayes_predicted,on = 't_docIdx', how='inner')
    classify_out = classify[['t_docIdx','t_labelIdx','label_id','predicted prob for doc']]
    classify_out['Check'] = classify_out['t_labelIdx'] == classify_out['label_id']
    accuracy = sum(classify_out.Check)*100.0/len(classify_out)
    return accuracy

# Test accuracy of Classifier

In [45]:
acc = Naive_Bayes_Classifier(df_test_data, False)
err = 100-acc
print("accuracy= %f"% (acc))
print("Naive Bayes Classifier Error = %f"% (err))

Accuracy= 80.756930
Naive Bayes Classifier Error = 19.243070


# Part F1)  Using Log(1+f)

In [ ]:
accuracy improvement by replacing frequency f of a word by log(1+f)

In [46]:
acc = Naive_Bayes_Classifier(df_test_data, True)
err = 100-acc
print("accuracy= %f"% (acc))
print("Error= %f"% (err))

Accuracy= 80.943497
Error= 19.056503


# Part F2) Using NLTK - removed stop words

In [47]:
# Since we know using log(1+f) is providing better accuracy let;s use it in conjunction with stop words removal
acc = Naive_Bayes_Classifier(df_test_data, True)
err = 100-acc
print("accuracy= %f"% (acc))
print("Error= %f"% (err))

Accuracy= 80.943497
Error= 19.056503


# accuracy improved by 2.5% after stop words removal